# 装袋和提升集成学习方法
* 集成学习：通过构建多个分类器并综合使用来完成学习任务，同时也被称为多分类器系统。其最大的特点就是结合各个弱分类器的长处。每一个弱分类器被称作「个体学习器」，从个体学习器类别来看，集成学习通常分为两种类型；从集成方式来看，也可以分为两类
    * 「同质」集成，在一个集成学习中，「个体学习器」是同一类型，如 「决策树集成」 所有个体学习器都为决策树
    * 「异质」集成，在一个集成学习中，「个体学习器」为不同类型，如一个集成学习中可以包含决策树模型也可以包含支持向量机模型
    * 并行式，当个体学习器之间不存在强依赖关系时，可同时生成并行化方法，其中代表算法为装袋（Bagging）算法
    * 串行式，当个体学习器之间存在强依赖关系时，必须串行生成序列化方法，其中代表算法为提升（Boosting）算法
* 结合策略：集成学习中，当数据被多个个体学习器学习后，通常有三种方法来最终决定学习结果
    * 平均法：在数值型输出中，最常用的结合策略为平均法（Averaging），在平均法中有两种方式：**简单平均法**、**加权平均法**
    * 投票法：对于分类输出而言，平均法显然效果不太好，最常用的结合策略为投票法（Voting），在投票法中主要有两种方式：**多数投票法**、**加权投票法**
    * 学习法：代表方法是 stacking ，当使用 stacking 的结合策略时， 不是对弱学习器的结果做简单的逻辑处理，而是再加上一层学习器，即把训练集弱学习器的学习结果作为输入，重新训练一个学习器来得到最终结果
* 装袋算法Bagging：装袋算法是并行式集成学习的代表，其原理也比较简单。算法步骤如下
    * 数据处理：将数据根据实际情况进行清洗整理
    * 随机采样：从样本中随机选出m个样本作为一个子样本集。有放回的重复T次，得到T个子样本集
    * 个体训练：设定T个个体学习器，将每一个子样本集放入对应个体学习器进行训练
    * 分类决策：用投票法集成进行分类决策
* 提升算法 Boosting：当「个体学习器」之间存在较强的依赖时，采用装袋的算法便有些不合适，此时最好的方法就是使用串行集成方式：提升（Boosting）。提升算法是可以将弱学习器提升为强学习器的算法，其具体思想是从初始训练集训练出一个「个体学习器」，再根据个体学习器的表现对训练样本分布进行调整，使得在个体学习器中判断错的训练样本在后续受到更多的关注，然后基于调整后的样本分布来训练下一个「个体学习器」。如此重复进行，直至个体学习器数目达到事先指定的值 T，最终将这 T 个「个体学习器」输出的值进行加权结合得到最终的输出值。


In [1]:
# Bagging Tree
# 以决策树作为弱分类器
import pandas as pd

data = pd.read_csv("course-14-student.csv", index_col=0)
data.head()

,school,sex,address,Pstatus,Pedu,reason,guardian,traveltime,studytime,schoolsup,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,0,0,0,0,1,2,2,1,1,0,...,3,2,3,0,0,2,6,2,2,2
1,0,0,0,1,2,2,0,0,1,1,...,4,2,2,0,0,2,4,2,2,2
2,0,0,0,1,2,0,2,0,1,0,...,3,2,1,1,2,2,10,2,2,3
3,0,0,0,1,0,1,2,0,2,1,...,2,1,1,0,0,4,2,3,3,1
4,0,0,0,1,0,1,0,0,1,1,...,3,2,1,0,1,4,4,2,3,3


In [2]:
# 划分数据集
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    data.iloc[:, :-1], data["G3"], test_size=0.3, random_state=35
)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((276, 26), (119, 26), (276,), (119,))

In [3]:
# 采用决策树方法先进行一次预测，作为对照组
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

dt_model = DecisionTreeClassifier(criterion="entropy", random_state=34)
dt_model.fit(X_train, y_train)  # 使用训练集训练模型

dt_y_pred = dt_model.predict(X_test)
accuracy_score(y_test, dt_y_pred)  # 计算使用决策树预测的准确率

0.8319327731092437

In [4]:
# 使用装袋（Bagging）的思想来提高预测准确率
'''
BaggingClassifier(base_estimator=None, n_estimators=10, max_samples=1.0, max_features=1.0)
    base_estimator: 表示基础分类器种类，默认为决策树
    n_estimators: 表示建立树的个数，默认为10
    max_samples: 表示从数据中选取训练样本的数量，int表示数量，float表示比例
    max_features: 表示抽取特征的数量，int表示数量，float表示比例
'''
from sklearn.ensemble import BaggingClassifier

tree = DecisionTreeClassifier(criterion="entropy", random_state=34)  # 使用决策树作为基学习器
bt_model = BaggingClassifier(tree, n_estimators=100, max_samples=1.0, random_state=3)

bt_model.fit(X_train, y_train)
bt_y_pred = bt_model.predict(X_test)
accuracy_score(y_test, bt_y_pred)  # 计算使用决策树预测的准确率

0.8907563025210085

In [12]:
# 随机森林，将一个大的数据集使用自助采样法进行处理
# 即从原样本数据集中随机抽取多个子样本集，并基于每一个子样本集生成相应的决策树。
# 这样，就可以构建出由许多小决策树组形成的决策树「森林」
# 所以，随机森林的名称来源就是「随机抽样 + 决策树森林」
'''
RandomForestClassifier(n_estimators, criterion, max_features, random_state=None)
    n_estimators: 表示建立树的个数，默认为10
    criterion: 表示特征划分方法选择，默认为 gini，可选择为 entropy (信息增益)
    max_features: 表示随机选择特征个数，默认为特征数的根号
'''
from sklearn.ensemble import RandomForestClassifier

# 这里构建 100 棵决策树，采用信息熵来寻找最优划分特征。
rf_model = RandomForestClassifier(
    n_estimators=100, max_features=None, criterion="entropy"
)

rf_model.fit(X_train, y_train)  # 进行模型的训练
rf_y_pred = rf_model.predict(X_test)
accuracy_score(y_test, rf_y_pred)

0.8907563025210085

In [13]:
# Adaboost
# AdaBoost（Adaptive Boosting）名为自适应增强，其主要自适应增强表现在：
# 上一个「个体学习器」中被错误分类的样本的权值会增大，正确分类的样本的权值会减小，并再次用来训练下一个基本分类器。
# 在每一轮迭代中，加入一个新的弱分类器，
# 直到达到某个预定的足够小的错误率或达到预先指定的最大迭代次数才确定最终的强分类器
'''
AdaBoostClassifier(base_estimators,n_estimators)
    base_estimators: 表示弱分类器种类，默认为CART分类树
    n_estimators: 表示弱学习器的最大个数，默认值为 50
'''
from sklearn.ensemble import AdaBoostClassifier

ad_model = AdaBoostClassifier(n_estimators=100)

ad_model.fit(X_train, y_train)
ad_y_pred = ad_model.predict(X_test)
accuracy_score(y_test, ad_y_pred)  # 计算使用决策树预测的准确率

/home/hanwen/anaconda3/envs/pentest/lib/python3.12/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


0.8151260504201681

In [17]:
# 梯度提升树GBDT
# 梯度提升树（Gradient Boosting Decison Tree，GBDT）同样是 Boosting 算法家族中的一员
# 梯度提升树所采用的是前向分布算法，且弱学习器限定了只能使用CART树模型
'''
GradientBoostingClassifier(max_depth = 3, learning_rate = 0.1, n_estimators = 100, random_state = None)
    max_depth: 表示生成 CART 树的最大深度，默认为 3
    learning_rate: 表示学习效率，默认为 0.1
    n_estimators: 表示弱学习器的最大个数，默认值为 100
    random_state: 表示随机数种子
'''
from sklearn.ensemble import GradientBoostingClassifier

gb_model = GradientBoostingClassifier(
    n_estimators=100, learning_rate=0.6, random_state=33
)

gb_model.fit(X_train, y_train)
gb_y_pred = gb_model.predict(X_test)
accuracy_score(y_test, gb_y_pred)  # 计算使用决策树预测的准确率

0.8823529411764706